**Image Caption Generator using ViT-GPT2 and Hugging Face Transformers**

**Project Overview**

This project demonstrates an end-to-end implementation of an **Image Captioning System** using a powerful **Vision-Encoder–Decoder architecture (ViT-GPT2)**. It combines the **visual understanding of Vision Transformers (ViT)** with the **text generation capabilities of GPT-2**, allowing the model to generate natural language descriptions for images.

Built using **Hugging Face Transformers** and deployed in a Google Colab environment, this notebook enables users to upload any image and receive an intelligent, human-like caption.

**Step 1: Install Dependencies**

transformers: for model loading and captioning

tokenizers: required by transformers

Pillow: for image processing

In [4]:
!pip uninstall -y tensorflow tensorflow-protobuf keras -q
!pip install transformers==4.37.2 tokenizers==0.14.1 Pillow nltk --no-deps -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.9 MB/s eta 0:00:00


** Step 2: Import Libraries**



In [5]:
# Import Libraries
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt


** Step 3: Load Pretrained Model**

Loads ViT-GPT2 image captioning model from Hugging Face.

ViTImageProcessor processes images.

AutoTokenizer prepares captions from model output.

In [6]:
# Load Model, Processor, Tokenizer
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

**Step 4: Set Device (GPU/CPU)**

In [7]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

**Step 5: Define Caption Generator Function**

Converts image to pixel values, feeds to model, decodes output into a caption.

In [8]:
#Define Caption Generator
def generate_caption(image, max_length=16, num_beams=4):
    image = image.convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt").pixel_values.to(device)
    output_ids = model.generate(inputs, max_length=max_length, num_beams=num_beams)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return caption

**Step 6: Upload Image in Colab**

Creates an upload button for users to provide images in the notebook.

In [9]:
# Upload image
upload = widgets.FileUpload(accept='image/*', multiple=False)
display(HTML("<h3> Upload image below:</h3>"))
display(upload)

FileUpload(value={}, accept='image/*', description='Upload')

** Step 7: Handle Image Generate Caption**

it’s displayed and captioned instantly.

In [10]:
# Button to trigger captioning
button = widgets.Button(description=" Generate Caption", button_style='primary')
output = widgets.Output()
display(button, output)

# Handle button click
def on_button_click(b):
    output.clear_output()
    if upload.value:
        file_info = list(upload.value.values())[0]
        image_data = file_info['content']
        image = Image.open(BytesIO(image_data))

        with output:
            # Show image
            display(image)

            # Generate and show caption
            caption = generate_caption(image)
            print(" Caption:", caption)
    else:
        with output:
            print("Please upload an image first.")

# Attach function to button
button.on_click(on_button_click)


Button(button_style='primary', description=' Generate Caption', style=ButtonStyle())

Output()

**Step 8 : Installing Gradio for Inteface**

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.3 MB/s eta 0:00:00


**Step 9 : Import gradio**

In [2]:
import gradio as gr

**Step 10 : Caption Generating using Gradio**

In [12]:
# Caption generation function
def generate_caption(image):
    image = image.convert("RGB") if image.mode != "RGB" else image
    inputs = feature_extractor(image, return_tensors="pt").pixel_values.to(model.device)
    output_ids = model.generate(inputs, max_length=16, num_beams=4)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Gradio interface
gr.Interface(fn=generate_caption, inputs=gr.Image(type="pil"), outputs="text", title="Image Captioning").launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7879013a2659291e7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## What I Learned

- Built an end-to-end image captioning system using Vision Transformer (ViT) and GPT-2.
- Learned how to integrate pre-trained models from Hugging Face for computer vision + NLP tasks.
- Gained hands-on experience with image processing, model inference, and generating natural language captions.
- Implemented interactive image upload in Colab.
- Improved my understanding of model architecture, tokenization, and deployment using tools like Gradio.
